In [1]:
from pyomo.environ import *
import numpy as np


In [3]:
m = ConcreteModel()

#sets and parameters
m.setMachine = Set(initialize=['A','B','C'])
m.Demand = 10000

#variables
m.C = Var(m.setMachine, bounds=(0,None))
m.P = Var(m.setMachine, within=Integers, bounds=(0,None))

#objective function
m.obj = Objective(expr = summation(m.C), sense=minimize)

#constraints
m.C1 = Constraint(expr = summation(m.P) == m.Demand)
m.C2 = Constraint(expr = m.C['A'] == 0.1*m.P['A']**2 + 0.5*m.P['A'] + 0.1)
m.C3 = Constraint(expr = m.C['B'] == 0.3*m.P['B'] + 0.5)
m.C4 = Constraint(expr = m.C['C'] == 0.01*m.P['C']**3)

#solve
opt = SolverFactory('ipopt')
m.results = opt.solve(m)

#print
m.pprint()

print('\n\nOF:',value(m.obj))

1 Set Declarations
    setMachine : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'A', 'B', 'C'}

2 Var Declarations
    C : Size=3, Index=setMachine
        Key : Lower : Value               : Upper : Fixed : Stale : Domain
          A :     0 : 0.10000000109177948 :  None : False : False :  Reals
          B :     0 :   2999.551316696256 :  None : False : False :  Reals
          C :     0 :  0.3162277710350839 :  None : False : False :  Reals
    P : Size=3, Index=setMachine
        Key : Lower : Value                  : Upper : Fixed : Stale : Domain
          A :     0 : 2.1983794637868577e-09 :  None : False : False : Integers
          B :     0 :      9996.837722320855 :  None : False : False : Integers
          C :     0 :      3.162277676947986 :  None : False : False : Integers

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
      

# Refomulated with binary variables

In [5]:
m = ConcreteModel()

#sets and parameters
m.setMachine = Set(initialize=['A','B','C'])
m.Demand = 10000
M = 1e6

#variables
m.C = Var(m.setMachine, bounds=(0,None))
m.P = Var(m.setMachine, within=Integers, bounds=(0,None))
m.B = Var(m.setMachine, within=Binary)

#objective function
m.obj = Objective(expr = summation(m.C), sense=minimize)

#constraints
m.C1 = Constraint(expr = summation(m.P) == m.Demand)
m.C2 = Constraint(expr = m.C['A'] == 0.1*m.P['A']**2 + 0.5*m.P['A'] + m.B['A']*0.1)
m.C3 = Constraint(expr = m.C['B'] == 0.3*m.P['B'] + m.B['B']*0.5)
m.C4 = Constraint(expr = m.C['C'] == 0.01*m.P['C']**3)

m.C5 = Constraint(expr = m.P['A'] <= m.B['A']*M)
m.C6 = Constraint(expr = m.P['B'] <= m.B['B']*M)

#solve
opt = SolverFactory('ipopt')
m.results = opt.solve(m)

#print
m.pprint()

print('\n\nOF:',value(m.obj))

1 Set Declarations
    setMachine : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'A', 'B', 'C'}

3 Var Declarations
    B : Size=3, Index=setMachine
        Key : Lower : Value                  : Upper : Fixed : Stale : Domain
          A :     0 : 4.8147764965956856e-08 :     1 : False : False : Binary
          B :     0 :   0.009996842734666153 :     1 : False : False : Binary
          C :     0 :                   None :     1 : False :  True : Binary
    C : Size=3, Index=setMachine
        Key : Lower : Value                : Upper : Fixed : Stale : Domain
          A :     0 : 8.82963519008844e-09 :  None : False : False :  Reals
          B :     0 :    2999.056314325285 :  None : False : False :  Reals
          C :     0 :  0.31622856160332996 :  None : False : False :  Reals
    P : Size=3, Index=setMachine
        Key : Lower : Value                 : Upper : Fixed : Stale : Domain
          A 